# Cumulative Prospect Theory

Note: Part of this is an adaption of https://towardsdatascience.com/an-introduction-to-bayesian-inference-in-pystan-c27078e58d53

Related documentation:
* https://mc-stan.org/docs/2_23/stan-users-guide/example-decision-analysis.html
* https://mc-stan.org/docs/2_25/stan-users-guide/examples.html

In [1]:
import os
import pickle
import pystan
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import multiprocessing
from scipy.special import expit
multiprocessing.set_start_method("fork")
sns.set()  # Nice plot aesthetic

In [2]:
# Set seed
np.random.seed(123)

### Utility of an outcome

The utility of a normalized outcome $x \in [0, 1]$ is defined as:
\begin{equation}
u(x) = x^{1 - \alpha}
\end{equation}
with
$\alpha \in [-\infty, 1]$
a free parameter describing the risk-aversion of the decision-maker (Holt & Laury, 2002). If $\alpha$
is positive,
$u''$
is negative, indicating risk-averse preferences (VNM, 1944). If
$\alpha$
is negative,
$u''$ 
is positive, indicating risk-seeking preferences. If
$\alpha$
is equal to 0,
$\forall x: u(x) = x$ and $u''(x) = 0$,
indicating risk-neutral preferences.

### Subjective probability perception 
$w$, the subjective probability perception function is defined following the formulation of Prelec (1998) as:
\begin{equation}
w(p) = e^{ - ( - \ln p )^{\beta}} 
\end{equation}
with $p\in (0, 1]$
the actual probability, and with $\beta \in (0, \infty)$,
a free parameter indicating the distortion of the probability
perception. We assume that $w(0) = 0$. For $\beta \in (0, 1)$,
the closer
$\beta$
is to zero, the more the small probabilities are overestimated, and the
large probabilities underestimated. For
$\beta=1$,
the subjective probabilities are the same as the objective
probabilities. For
$\beta \in (1, \infty)$,
the higher
$\beta$,
the more the small probabilities are underestimated, and the large
probabilities overestimated.

### Subjective expected utility

The subjective expected utility of a lottery $L = \{(p_1, x_1), \dots, (p_n, x_n)\}$ is defined as:
\begin{equation}
SEU(L) = \sum_{i=1}^{n} \pi_i u(x_i)
\end{equation}
where $\pi_i$(for gains) is:
\begin{equation}
\pi_i = w\left(\sum_{k=i}^{n} p_k \right) - w\left(\sum_{k=i+1}^{n} p_k\right)
\end{equation}

### Probability to choose a lottery

The probability to choose the lottery $L_k \in \mathcal{L} = \{L_1, \dots, L_{N}\}$ is defined as: 
\begin{equation}
P(L_k) = \frac {e^{SEU(L_k) / \tau}}{\sum _{i=1}^{N} e^{SEU(L_i) / \tau } }
\end{equation}
$\tau \in (0, \infty)$
a free parameter describing to which extent decision-making is stochastic.
The higher
$\tau$,
the more the decision-making is stochastic.

### Generate data

Note: We consider here a special case where $L = \{(p, x)\}$ (one lottery has one possible positive output).

In [3]:
def generate_stimuli(n_trial, n_lot):
    
    possible_p = [0.25, 0.5, 0.75, 1]
    possible_x = np.arange(1, 4)
    p = np.random.random(size=(n_trial, 2))
    x = np.array([np.random.choice(possible_x, size=2, replace=False) for _ in range(n_trial)])
    x = -np.sort(-x, axis=-1)
    return pd.DataFrame({"p0": p[:, 0], "p1": p[:, 1], "x0": x[:, 0], "x1": x[:, 1]})

In [4]:
def simulate(param, n_trial):

    data = generate_stimuli(n_trial=n_trial)
    
    alpha, tau, beta = param

    n = len(data)

    sp0 = np.exp(-(-np.log(data.p0.values)) ** beta)
    sp1 = np.exp(-(-np.log(data.p1.values)) ** beta)
    
    su0 = data.x0.values ** (1 - alpha)
    su1 = data.x1.values ** (1 - alpha)
    
    v0 = sp0 * su0
    v1 = sp1 * su1
    
    delta = v0 - v1
    
    p = np.zeros((n, 2))
    p[:, 0] = expit(delta/tau)
    p[:, 1] = 1 - p[:, 0]
    
    c = np.zeros(n, dtype=int)
    r = np.random.random(size=n)
    c[:] = p[:, 1] > r
    
    data["c"] = c
    return data

In [5]:
# True parameters
alpha_true = 0.3
tau_true = 0.1
beta_true = 0.8

# Number of trials
n_trial = 1000

# Generate parameters to simualte
param_true = alpha_true, tau_true, beta_true

# Simulate
d = simulate(param=param_true, n_trial=n_trial)
d

,p0,p1,x0,x1,c
0,0.25,1.0,3,2,1
1,0.50,1.0,2,1,0
2,0.25,1.0,3,1,1
3,0.25,1.0,3,2,1
4,0.75,1.0,3,2,1
...,...,...,...,...,...
995,0.25,0.5,3,2,1
996,0.25,1.0,3,1,1
997,0.25,0.5,3,1,0
998,0.50,1.0,2,1,1


In [6]:
# True parameters
alpha_true = 0.3
tau_true = 0.1

# Number of trials
n_trial = 1000

# Number of lottery per trial
n_lot = 2

p = np.random.random(size=(n_lot, n_trial))
x = np.random.random(size=(n_lot, n_trial))

u = x**(1-alpha_true)
eu = p * u

p_choice = np.exp(eu/tau_true)
p_choice /= p_choice.sum(axis=0)

y = np.zeros(n_trial, dtype=int)
y[:] = p_choice[1] > np.random.random(size=n_trial)

data = {'N': n_trial, 'x0': x[0], 'x1': x[1], 'p0': p[0], 'p1': p[1], 'y': y}
pd.DataFrame(data)

,N,x0,x1,p0,p1,y
0,1000,0.549577,0.242381,0.674709,0.152820,0
1,1000,0.613223,0.458828,0.735215,0.431809,0
2,1000,0.841033,0.514185,0.962716,0.357361,0
3,1000,0.825513,0.167835,0.216763,0.983442,0
4,1000,0.782138,0.239888,0.514050,0.563739,0
...,...,...,...,...,...,...
995,1000,0.569042,0.728209,0.379095,0.024971,0
996,1000,0.788716,0.342232,0.026529,0.690979,1
997,1000,0.250664,0.000489,0.372233,0.273774,1
998,1000,0.348286,0.018089,0.565436,0.883045,1


## Inference using Stan

### Define the model

Note: in Stan language, vector indexes start from 1 (unlike Python thats starts from 0) 

In [ ]:
model = """
functions {
    vector w(vector p, real gam){
        int p_len = dims(p)[1];
        vector[p_len] w;
        for(i in 1:p_len){
            real p_i = p[i];
            //Yang 2017 w-function eq(5)
            // ret[ii] = p_i^gam / ( p_i^gam + (1-p_i)^gam )^(1 / gam);
            w[ii] = p_i^gam / ( p_i^gam + (1-p_i)^gam )^(1 / gam);
    }
    return w;
  }
  real w_real(real p, real gam){
    real ret = p^gam / ( p^gam + (1-p)^gam )^(1 / gam);
    return(ret);
  }
  vector prob_distort(vector p, real gam){
    int p_len = dims(p)[1];
    vector[p_len] pi_p;
    for(kk in 1:(p_len-1)){
      pi_p[kk] = w_real(sum(p[kk:p_len]), gam) - w_real(sum(p[(kk+1):p_len]), gam);
    }
    pi_p[p_len] = 1 - sum(pi_p[1:(p_len-1)]);
    return(pi_p);
  }
  vector value_func(vector x, real alpha, real lambda, real eta){
    int N_val = dims(x)[1];
    vector[N_val] ret;
    for(ii in 1:N_val){
      real xi = x[ii];
      if(xi < 0){
        ret[ii] = - lambda * (-xi)^eta;
      }else{
        ret[ii] = xi^alpha;
      }
    }
    return(ret);
  }
}
data {
  int<lower = 1> TT; //Number of choices
  int<lower = 1> N_lotto;
  int<lower = 1> N_choices_max; //Max number of choices
  int N_choices[N_lotto];
  int<lower = 1, upper = N_lotto> choices[TT]; //the actual choices
  matrix[N_lotto, N_choices_max] lotto_probabilities;
  matrix[N_lotto, N_choices_max] lotto_rewards;
}
parameters {
  real<lower=0, upper=1> alpha;
  real<lower=0, upper=1> eta;
  real<lower=1, upper=2> lambda;
  real<lower=0, upper=1> gamma;
}
transformed parameters{
  simplex[N_lotto] p;
  vector[N_lotto] CPV;
  
  for(ii in 1:N_lotto){
    int n_choices_lotto_i = N_choices[ii];
    vector[n_choices_lotto_i] probs_i = to_vector(lotto_probabilities[ii, 1:n_choices_lotto_i]);
    vector[n_choices_lotto_i] rewards_i = to_vector(lotto_rewards[ii, 1:n_choices_lotto_i]);
    CPV[ii] =  sum(prob_distort( probs_i, gamma) .* value_func( rewards_i, alpha, lambda, eta)); //subjective expectation
  }
  p = softmax(CPV);
}
model {
  alpha ~ beta(1,1);
  eta ~ beta(1,1);
  lambda ~ uniform(1,2);
  gamma ~ beta(1,1);
  for(ii in 1:TT){
    target += log(p[choices[ii]]);
  }
}
"""

In [7]:
model = """
functions {
    vector w(vector p, real gam){
        int p_len = dims(p)[1];
        vector[p_len] w;
        for(i in 1:p_len){
            real p_i = p[i];
            //Yang 2017 w-function eq(5)
            // ret[ii] = p_i^gam / ( p_i^gam + (1-p_i)^gam )^(1 / gam);
            w[ii] = p_i^gam / ( p_i^gam + (1-p_i)^gam )^(1 / gam);
    }
    return w;
  }
  real w_real(real p, real gam){
    real ret = p^gam / ( p^gam + (1-p)^gam )^(1 / gam);
    return(ret);
  }
  vector prob_distort(vector p, real gam){
    int p_len = dims(p)[1];
    vector[p_len] pi_p;
    for(kk in 1:(p_len-1)){
      pi_p[kk] = w_real(sum(p[kk:p_len]), gam) - w_real(sum(p[(kk+1):p_len]), gam);
    }
    pi_p[p_len] = 1 - sum(pi_p[1:(p_len-1)]);
    return(pi_p);
  }
  vector value_func(vector x, real alpha, real lambda, real eta){
    int N_val = dims(x)[1];
    vector[N_val] ret;
    for(ii in 1:N_val){
      real xi = x[ii];
      if(xi < 0){
        ret[ii] = - lambda * (-xi)^eta;
      }else{
        ret[ii] = xi^alpha;
      }
    }
    return(ret);
  }
}
data {
  int<lower = 1> TT; //Number of choices
  int<lower = 1> N_lotto;
  int<lower = 1> N_choices_max; //Max number of choices
  int N_choices[N_lotto];
  int<lower = 1, upper = N_lotto> choices[TT]; //the actual choices
  matrix[N_lotto, N_choices_max] lotto_probabilities;
  matrix[N_lotto, N_choices_max] lotto_rewards;
}
parameters {
  real<lower=0, upper=1> alpha;
  real<lower=0, upper=1> eta;
  real<lower=1, upper=2> lambda;
  real<lower=0, upper=1> gamma;
}
transformed parameters{
  simplex[N_lotto] p;
  vector[N_lotto] CPV;
  
  for(ii in 1:N_lotto){
    int n_choices_lotto_i = N_choices[ii];
    vector[n_choices_lotto_i] probs_i = to_vector(lotto_probabilities[ii, 1:n_choices_lotto_i]);
    vector[n_choices_lotto_i] rewards_i = to_vector(lotto_rewards[ii, 1:n_choices_lotto_i]);
    CPV[ii] =  sum(prob_distort( probs_i, gamma) .* value_func( rewards_i, alpha, lambda, eta)); //subjective expectation
  }
  p = softmax(CPV);
}
model {
  alpha ~ beta(1,1);
  eta ~ beta(1,1);
  lambda ~ uniform(1,2);
  gamma ~ beta(1,1);
  for(ii in 1:TT){
    target += log(p[choices[ii]]);
  }
}
"""

### Compile the model

In [8]:
# Put it to true if you edit the model
force_compilation = True

# Where to save backup
bkp_folder = 'bkp'
os.makedirs(bkp_folder, exist_ok=True)
bkp_file = os.path.join(bkp_folder, 'cpt_model.pkl')

if not os.path.exists(bkp_file) or force_compilation is True:
    
    # Compile the model
    sm = pystan.StanModel(model_code=model)
    
    # Save the model
    with open(bkp_file, 'wb') as f:
        pickle.dump(sm, f)
else:
    # Load the model
    sm = pickle.load(open(bkp_file, 'rb'))

ValueError: Failed to parse Stan model 'anon_model_c0c90454372f58626ca4a5c420cb1f69'. Error message:
SYNTAX ERROR, MESSAGE(S) FROM PARSER:
Variable "ii" does not exist.
 error in 'unknown file name' at line 10, column 16
  -------------------------------------------------
     8:             //Yang 2017 w-function eq(5)
     9:             // ret[ii] = p_i^gam / ( p_i^gam + (1-p_i)^gam )^(1 / gam);
    10:             w[ii] = p_i^gam / ( p_i^gam + (1-p_i)^gam )^(1 / gam);
                       ^
    11:     }
  -------------------------------------------------



### Sampling

* `iter`: number of samples that will be generated from each Markov chain.
* `chains`: number of chains from which samples will be combined to form the posterior distribution. Because the underlying Markov process is stochastic, it's advantageous to have more chains that will initialise at different locations in parameter space, though adding more chains will increase the amount of time it takes to sample. 
* `warmup` (also known as 'burn-in'): number of samples that will be discarded from the beginning of each chain. As the early samples will be drawn when the Markov chain hasn't had a chance to reach equilibrium. By default this is half of iter, so for each chain we'll get 1000 samples, and chuck away the first 500. With 4 chains, we'll have 2000 samples in total.
* `thin`: interval in sampling at which samples are retained. E.g.: if thin is equal to 3, every third sample is retained and the rest are discarded. This can be necessary to mitigate the effect of correlation between successive samples. It's set to 1 here, and so every sample is retained. 
* `seed`: Seed for the random generator. It allows for reproducibility.

In [ ]:
# Train the model and generate samples
fit = sm.sampling(data=data, iter=1000, chains=4, warmup=500, thin=1, seed=101)
fit

In [ ]:
# Cast the fit output to a pandas DataFrame
summary_dict = fit.summary()
df = pd.DataFrame(summary_dict['summary'], 
                  columns=summary_dict['summary_colnames'], 
                  index=summary_dict['summary_rownames'])
df

### Plot the results

In [ ]:
# Select a subset of samples to plot
n = 1000
idx = np.random.choice(range(len(alpha)), size=n, replace=False)

In [ ]:
# Utility function
def f(x, alpha):
    return x**(1-alpha)

# Extract mean
alpha_mean = df['mean']['alpha']

# Extrac trace
alpha = fit['alpha']

# Create fig
fig, ax = plt.subplots(figsize=(10, 6))

# Set limits
x_min, x_max = 0, 1
y_min, y_max = 0, 1

# Generate x-values
x = np.linspace(x_min, x_max, 100)

# Plot a subset
for i in idx:
    ax.plot(x, f(x, alpha[i]), color='lightsteelblue', alpha=0.005 )

# Plot mean
ax.plot(x, f(x, alpha_mean), label="fit")

# Plot truth
ax.plot(x, f(x, alpha_true), ls=':', label='true', color="red")

# Plot bounds
ax.plot(x, f(x, -1), color='0.01', ls='--', lw=0.5, label="bounds")
ax.plot(x, f(x, 0.5), color='0.01', ls='--', lw=0.5)

# Pimp your plot
ax.set_xlabel('x')
ax.set_ylabel('u(x)')
ax.set_title('Fitted Utility function')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.legend()

plt.show()

In [ ]:
# Softmax function given the difference of value between 2 options
def f(x, tau):
    return expit(x/tau)

# Extract mean
beta_mean = df['mean']['tau']

# Extrac trace
beta = fit['tau']

# Create fig
fig, ax = plt.subplots(figsize=(10, 6))

# Set limits
x_min, x_max = -1, 1
y_min, y_max = 0, 1

# Generate x-values
x = np.linspace(x_min, x_max, 100)

# Plot a subset
for i in idx:
    ax.plot(x, f(x, tau[i]), color='lightsteelblue', alpha=0.005)

# Plot mean
ax.plot(x, f(x, tau_mean), label="fit")

# Plot truth
ax.plot(x, f(x, tau_true), ls=':', label='true', color="red")

# Pimp your plot
ax.set_xlabel('$EU(L_1) - EU(L_2)$')
ax.set_ylabel('$P(L_1)$')
ax.set_title('Fitted Softmax function')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)

ax.legend()
plt.show()

In [ ]:
def f(x, beta):
    return np.exp(-(-np.log(x) ** beta)

# Create fig
fig, ax = plt.subplots(figsize=(10, 6))

# Set limits
x_min, x_max = 0, 1
y_min, y_max = 0, 1

# Generate x-values
x = np.linspace(x_min, x_max, 100)

# Plot a subset
for i in idx:
    ax.plot(x, f(x, beta[i]), color='lightsteelblue', alpha=0.005)

# Plot mean
ax.plot(x, f(x, beta_mean), label="fit")

# Plot truth
ax.plot(x, f(x, beta_true), ls=':', label='true', color="red")

# Pimp your plot
ax.set_xlabel('$p$')
ax.set_ylabel('$w(p)$')
ax.set_title('Fitted subjective probability perception function')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)

ax.legend()
plt.show()

In [ ]:
for param_name in ("alpha", "tau", "beta"):

    # Extract trace
    param = fit[param_name]
    
    # Summary statistics
    mean = np.mean(param)
    median = np.median(param)
    cred_min, cred_max = np.percentile(param, 2.5), np.percentile(param, 97.5)

    # Plotting
    fig, axes = plt.subplots(ncols=2, figsize=(10, 5))
    
    ax = axes[0]
    
    ax.plot(param)
    ax.set_xlabel('samples')
    ax.set_ylabel(param_name)
    ax.axhline(mean, color='r', lw=2, linestyle='--')
    ax.axhline(median, color='c', lw=2, linestyle='--')
    ax.axhline(cred_min, linestyle=':', color='k', alpha=0.2)
    ax.axhline(cred_max, linestyle=':', color='k', alpha=0.2)
    ax.set_title('Trace and Posterior Distribution for {}'.format(param_name))

    ax = axes[1]
    ax.hist(param, 30, density=True); sns.kdeplot(param, shade=True)
    ax.set_xlabel(param_name)
    ax.set_ylabel('density')
    ax.axvline(mean, color='r', lw=2, linestyle='--',label='mean')
    ax.axvline(median, color='c', lw=2, linestyle='--',label='median')
    ax.axvline(cred_min, linestyle=':', color='k', alpha=0.2, label='95% CI')
    ax.axvline(cred_max, linestyle=':', color='k', alpha=0.2)
    ax.legend()
    
    plt.tight_layout()

## Using stochastic gradient descent

In [ ]:
EPS = np.finfo(float).eps

def objective(param, data):
    
    alpha, tau, beta = param

    n = len(data)

    sp0 = np.exp(-(-np.log(data.p0.values)) ** beta)
    sp1 = np.exp(-(-np.log(data.p1.values)) ** beta)
    
    su0 = data.x0.values ** (1 - alpha)
    su1 = data.x1.values ** (1 - alpha)
    
    v0 = sp0 * su0
    v1 = sp1 * su1
    
    delta = v0 - v1
    
    p = np.zeros((2, n))
    p[0] = expit(delta/tau)
    p[1] = 1 - p[0]

    lls = np.log(p[0, data.c.values==0] + EPS).sum()
    lls += np.log(p[1, data.c.values==1] + EPS).sum()
    # Since we will look for the minimum, let's return -LLS instead of LLS
    return -lls

In [ ]:
# Define bounds and an initial guess
bounds = (-1, 0.5), (0.0001, None), (0, None) 
init_guess = (0, 1)

# Run the optimizer
res = scipy.optimize.minimize(
    fun=objective,
    x0=init_guess,
    bounds=bounds,
    args=(data, ))
res

A '[OptimizeResult](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html#scipy.optimize.OptimizeResult)' is returned. It contains:
* `fun` (NumPy array): Value of objective function.
* `hess_inv` (object): Inverse of the objective function’s Hessian; may be an approximation. Not available for all solvers. The type of this attribute may be either np.ndarray or scipy.sparse.linalg.LinearOperator. Here, it is a scipy.sparse.linalg.LinearOperator.
* `jac` (NumPy array): Value of the Jacobian.
* `nfev` (int): Number of evaluations of the objective functions.
* `message` (str): Description of the cause of the termination.
* `nit` (int): Number of iterations performed by the optimizer.
* `njev` (int): Number of evaluations of the objective functions and of its Jacobian.
* `status` (int): Termination status of the optimizer. Its value depends on the underlying solver. Refer to message for details.
* `success` (bool): Whether or not the optimizer exited successfully.
* `x` (NumPy array): the solution of the optimization.

In [ ]:
# Extract the best param and best value 
best_param = res.x
best_value = res.fun

print("Estimation parameters: ", best_param)
print("LLS: ", - best_value)

In [ ]:
alpha_est, tau_est, beta_est = res.x

In [ ]:
# Utility function
def f(x, alpha):
    return x**(1-alpha)

# Create fig
fig, ax = plt.subplots(figsize=(10, 6))

# Set limits
x_min, x_max = 0, 1
y_min, y_max = 0, 1

# Generate x-values
x = np.linspace(x_min, x_max, 100)

# Plot estimate
ax.plot(x, f(x, alpha_est), label="fit")

# Plot truth
ax.plot(x, f(x, alpha_true), ls=':', label='true', color="red")

# Plot bounds
ax.plot(x, f(x, -1), color='0.01', ls='--', lw=0.5, label="bounds")
ax.plot(x, f(x, 0.5), color='0.01', ls='--', lw=0.5)

# Pimp your plot
ax.set_xlabel('x')
ax.set_ylabel('u(x)')
ax.set_title('Fitted Utility function')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.legend()

plt.show()

In [ ]:
# Softmax function given the difference of value between 2 options
def f(x, tau):
    return expit(x/tau)

# Create fig
fig, ax = plt.subplots(figsize=(10, 6))

# Set limits
x_min, x_max = -1, 1
y_min, y_max = 0, 1

# Generate x-values
x = np.linspace(x_min, x_max, 100)

# Plot estimate
ax.plot(x, f(x, tau_est), label="fit")

# Plot truth
ax.plot(x, f(x, tau_true), ls=':', label='true', color="red")

# Pimp your plot
ax.set_xlabel('$EU(L_1) - EU(L_2)$')
ax.set_ylabel('$P(L_1)$')
ax.set_title('Fitted Softmax function')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.legend()

plt.show()

In [ ]:
# Probability perception function
def f(x, beta):
    return np.exp(-(-np.log(x) ** beta)

# Create fig
fig, ax = plt.subplots(figsize=(10, 6))

# Set limits
x_min, x_max = 0, 1
y_min, y_max = 0, 1

# Generate x-values
x = np.linspace(x_min, x_max, 100)

# Plot mean
ax.plot(x, f(x, beta_est), label="fit")

# Plot truth
y_true = f(x, beta_true)
ax.plot(x, y_true, ls=':', label='true', color="red")

# Pimp your plot
ax.set_xlabel('$p$')
ax.set_ylabel('$w(p)$')
ax.set_title('Fitted subjective probability perception function')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)

ax.legend()
plt.show()